In [1]:
import re
import os
import pandas as pd
import numpy as np
import math
import random
import jieba
import sys
from sklearn.naive_bayes import GaussianNB
from sklearn import svm
from sklearn import metrics
from sklearn import preprocessing

In [2]:
print(sys.version)

3.8.2 (v3.8.2:7b3ab5921f, Feb 24 2020, 17:52:18) 
[Clang 6.0 (clang-600.0.57)]


In [3]:
def word2idx(embedding):
    wordidx = {}
    for i in range(len(embedding)):
        wordidx[embedding['name'][i]] = i
    return(wordidx)
    
def sen2idx(sen, wordidx):
    idxword = []
    for s in sen:
        try:
            idx = wordidx[s]
            idxword.append(idx)
        except:
            continue
    return(idxword)

In [4]:
os.chdir(r'Replication_TrainingDatasetCensorship/') # set working directory

In [5]:
jieba.load_userdict("auxiliary/Dict.txt")

Building prefix dict from the default dictionary ...
Dumping model to file cache /var/folders/8g/74cy_pd90b94x2b6lxyf92v40000gn/T/jieba.cache
Loading model cost 0.692 seconds.
Prefix dict has been built successfully.


In [6]:
words =[# freedom
    "自由", "言论自由", "集会自由", "新闻自由", "结社自由", "自由权", "民主自由", "自由言论", "创作自由", 
    "婚姻自主", "自由民主", "自由市场", "自决", "自决权", "生而自由", "自由自在", "自由选择", "自由思想", 
    "公民自由", "自由竞争", "宗教自由", "自由价格", 
    # democracy
    "民主", "自由民主", "民主自由", "民主制度", "民主化", "社会民主主义", "民主运动", "民主主义", "民主改革", 
    "民主制", "民主选举", "民主权力", "多党制", "民主法制", "民主权利" ,
    # election
    "选举", "直接选举", "议会选举", "间接选举", "直选", "换届选举", "民选", "投票选举", "全民公决", "总统大选", 
    "大选", "普选", "全民投票", "民主选举",
    # collective action
    "抗议", "示威", "示威游行", "示威抗议", "游行示威", "静坐示威", "绝食抗议", "请愿", "示威运动", "游行", 
    "罢教", "静坐", "集会游行", "罢课", "签名运动", 
    # negative figures
    "林彪", "王洪文", "张春桥", "江青", "姚文元", "刘晓波", "丹增嘉措", "李洪志", "陈水扁", "黎智英", "艾未未", 
    "李登辉", "李柱铭", "何俊仁", "陈方安生", "达赖", "魏京生",
    # social control
    "维稳", "处突", "社会治安", "反恐怖", "公安工作", "预防犯罪", "收容审查", "治安工作", "大排查", "扫黄打非", 
    "接访", "反邪教",
    # surveillance
    "监控", "监测", "监视", "管控", "监看", "监视系统", "截听", "监控中心", "情报服务", "排查", "监视器",
    "情报搜集", "间谍卫星", "管理网络", "监控器", "监控站", "监控室", "数据采集", 
    # CCP
    "党中央",  "中国共产党", "党支部", "中共中央", "共青团", "共青团中央", "党委", "中央党校",
    # historical events
    "抗日战争", "解放战争", "抗美援朝", "改革开放", "香港回归", "长征", "三大战役", "秋收起义", "南昌起义", "志愿军", 
    "土地改革", "六四", "遵义会议", "广州起义", "百团大战", "文革", "文化大革命", "大跃进", "四人帮",
    # positive figures
    "毛泽东", "江泽民", "胡锦涛", "习近平", "周恩来", "朱镕基", "温家宝", "李克强", "邓小平", "曾庆红", "华国锋", "李鹏", 
    "杨尚昆", "谷牧", "吴邦国", "李岚清", "纪登奎", "乔石","邹家华", "李瑞环", "俞正声", "张高丽", "田纪云", "回良玉", 
    "李源潮", "贾庆林", "姚依林", "张立昌", "尉健行", "姜春云", "李铁映", "王兆国", "罗干", "杨汝岱", "王光英", "彭佩云", 
    "刘云山", "丁关根", "彭真", "胡启立", "曾培炎", "何东昌"]

len(words)

182

# Baidubaike

In [8]:
# remove punctuation, news agencies, duplicate headlines, numbers & English letters
# test set
news = pd.read_csv('auxiliary/Political_news_titles.csv')
news = news[news['label'].isnull()==False]
punct = ['......', '，', '--', '：',  '丨', '「', '」', '、', '——',  '-',  '_', '...', '·', '。',
        '【', '】', '《', '》', '/', '(图)', '(组图)', '[图]', '‧', '.', ':', ',', '（下）', '（上）', '（图）', '（组图）', ' ',
        '丨', '|', '(', ')', '（', '）', '[', ']', "'", '’', '‘', '“', '”', '%', '?', '!']
for i in range(len(news)):
    text = news.iloc[i,0]
    for p in punct:
      text = text.replace(p, "")
    text = re.sub('[a-zA-Z]','',text)
    text = re.sub("\d+", "", text)
    news.iloc[i,0] = text
news = news[news['label']!=0]
news.drop_duplicates(subset=['sentence'], inplace=True)
news = news.dropna(how='any',axis=0) 
news = news.reset_index(drop=True)
news["label"] = news["label"].astype(int)
news.head()

,sentence,label
0,达里奥资本市场已不再自由贫富差距达年代以来最严重,-1
1,新闻自由日联合国秘书长强调记者为新冠大流行错误信息提供,-1
2,一组老照片自由女神像是如何诞生的,1
3,在互联网自由日抗议苹果公司同中共合作审查网络,-1
4,香港居民权利自由的坚实保障,1


In [9]:
# training set
train = pd.read_csv('auxiliary/TNEWS_subset.csv')
train = train[train['label'].isnull()==False]
for i in range(len(train)):
    text = train.iloc[i,0]
    for p in punct:
      text = text.replace(p, "")
    text = re.sub('[a-zA-Z]','',text)
    text = re.sub("\d+", "", text)
    train.iloc[i,0] = text
train = train[train['label']!=0]
train.drop_duplicates(subset=['sentence'], inplace=True)
train = train.dropna(how='any',axis=0) 
train = train.reset_index(drop=True)
len(train)

2420

In [10]:
# load Baidubaike word embedding
bd = pd.read_csv('output/2.dropna_baidubaike_clean.csv')
bd.rename(columns={'Unnamed: 0': 'name'}, inplace=True)
bd_wordidx = word2idx(bd)
bd.head()

,name,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V292,V293,V294,V295,V296,V297,V298,V299,V300,V301
0,，,-0.242002,0.139317,0.073788,-0.191109,0.088181,0.130866,0.126029,-0.072774,0.033516,...,0.011133,0.172242,0.198639,0.049223,-0.054923,-0.083077,-0.085516,0.141039,0.055923,-0.080048
1,的,-0.086717,0.077708,0.095153,-0.077534,-0.201572,-0.039050,0.224178,-0.177118,-0.027403,...,-0.000238,0.170862,0.145751,0.109331,0.044553,0.032508,-0.257849,0.111964,0.030822,-0.128930
2,。,-0.114365,0.122019,0.028330,-0.154405,-0.082406,0.006950,0.244836,-0.022653,0.054961,...,0.150479,0.037960,0.077362,0.077516,0.042213,0.055036,0.039008,0.110687,0.036073,-0.137635
3,、,-0.218755,0.104672,-0.032240,-0.175267,-0.031778,0.085487,0.042656,-0.130028,0.323578,...,0.092067,0.191939,-0.080064,-0.140970,-0.223796,0.046355,-0.174176,-0.062020,-0.004782,0.213918
4,",",-0.166416,0.458875,0.117787,-0.300892,-0.232379,0.109313,-0.325269,-0.166099,-0.742212,...,-0.099326,0.021598,-0.142140,0.352612,-0.080029,0.281831,-0.281419,0.442771,0.091643,-0.376297


In [11]:
train_embedding = pd.DataFrame(index=range(0, len(train)), columns=range(0, 300))
test_embedding = pd.DataFrame(index=range(0, len(news)), columns=range(0, 300))

# convert headlines in training set to word embedding
for i in range(0, len(train)):
    text = jieba.lcut(train.iloc[i,0])
    textidx = sen2idx(text, bd_wordidx)
    train_embedding.loc[i] = bd.loc[textidx,'V2':'V301'].mean(axis=0).values
    if i%1000==0:
        print(i)
        
# convert headlines in test set to word embedding
for i in range(0, len(news)):
    text = jieba.lcut(news.iloc[i,0])
    textidx = sen2idx(text, bd_wordidx)
    test_embedding.loc[i] = bd.loc[textidx,'V2':'V301'].mean(axis=0).values
    if i%2500==0:
        print(i)

0
1000
2000
0
2500
5000
7500


In [12]:
train_lab = train['label'][train_embedding.isnull().any(axis=1)==False].values
train_embedding = train_embedding.loc[train_embedding.isnull().any(axis=1)==False]
test_lab = news['label'][test_embedding.isnull().any(axis=1)==False].values
news = news.loc[test_embedding.isnull().any(axis=1)==False]
test_embedding = test_embedding.loc[test_embedding.isnull().any(axis=1)==False]
train_bd = preprocessing.scale(train_embedding)
test_bd = preprocessing.scale(test_embedding)

In [13]:
clf = svm.SVC()
clf.fit(train_bd,train_lab)
y_pred=clf.predict(test_bd)
print("Accuracy:",metrics.accuracy_score(test_lab, y_pred))

Accuracy: 0.7759322406775933


In [14]:
for i in range(len(news)):
    news.iloc[i, 0] = ' ' + ' '.join(jieba.lcut(news.iloc[i, 0]))+' '

# store prediction
res = pd.DataFrame(index=range(5000), columns=range(20))
category = [range(0, 22), range(22, 37), range(37, 51), range(51, 66), range(66,83), 
            range(83, 95), range(95, 113), range(113, 121), range(121, 140), range(140, 182)]
for n, c in enumerate(category):
    counter = 0
    for w in c:
        counter += 1
        word = ' ' + words[w] + ' '
        id = [i for i in range(len(news)) if word in news.iloc[i, 0]]
        dif = (y_pred[id] - test_lab[id])
        start = 5000 - res.iloc[:,n].isnull().sum()
        end = start + len(dif.tolist())
        res.iloc[start:end, n] = dif.tolist()
        res.iloc[start:end, n+10] = counter
res.to_csv(r'output/svm_bd.csv', header=False, index=False)

In [15]:
clf = GaussianNB()
clf.fit(train_bd,train_lab)
y_pred=clf.predict(test_bd)
print("Accuracy:",metrics.accuracy_score(test_lab, y_pred))

Accuracy: 0.7688923110768893


In [16]:
res = pd.DataFrame(index=range(5000), columns=range(20))
category = [range(0, 22), range(22, 37), range(37, 51), range(51, 66), range(66,83), 
            range(83, 95), range(95, 113), range(113, 121), range(121, 140), range(140, 182)]
for n, c in enumerate(category):
    counter = 0
    for w in c:
        counter += 1
        word = ' ' + words[w] + ' '
        id = [i for i in range(len(news)) if word in news.iloc[i, 0]]
        dif = (y_pred[id] - test_lab[id])
        start = 5000 - res.iloc[:,n].isnull().sum()
        end = start + len(dif.tolist())
        res.iloc[start:end, n] = dif.tolist()
        res.iloc[start:end, n+10] = counter
res.to_csv(r'output/nb_bd.csv', header=False, index=False)

# Wikipedia

In [17]:
news = pd.read_csv('auxiliary/Political_news_titles.csv')
news = news[news['label'].isnull()==False]
punct = ['......', '，', '--', '：',  '丨', '「', '」', '、', '——',  '-',  '_', '...', '·', '。',
        '【', '】', '《', '》', '/', '(图)', '(组图)', '[图]', '‧', '.', ':', ',', '（下）', '（上）', '（图）', '（组图）', ' ',
        '丨', '|', '(', ')', '（', '）', '[', ']', "'", '’', '‘', '“', '”', '%', '?', '!']
for i in range(len(news)):
    text = news.iloc[i,0]
    for p in punct:
      text = text.replace(p, "")
    text = re.sub('[a-zA-Z]','',text)
    text = re.sub("\d+", "", text)
    news.iloc[i,0] = text
news = news[news['label']!=0]
news.drop_duplicates(subset=['sentence'], inplace=True)
news = news.dropna(how='any',axis=0) 
news = news.reset_index(drop=True)
news["label"] = news["label"].astype(int)
news.head()

,sentence,label
0,达里奥资本市场已不再自由贫富差距达年代以来最严重,-1
1,新闻自由日联合国秘书长强调记者为新冠大流行错误信息提供,-1
2,一组老照片自由女神像是如何诞生的,1
3,在互联网自由日抗议苹果公司同中共合作审查网络,-1
4,香港居民权利自由的坚实保障,1


In [18]:
train = pd.read_csv('auxiliary/TNEWS_subset.csv')
train = train[train['label'].isnull()==False]
for i in range(len(train)):
    text = train.iloc[i,0]
    for p in punct:
      text = text.replace(p, "")
    text = re.sub('[a-zA-Z]','',text)
    text = re.sub("\d+", "", text)
    train.iloc[i,0] = text
train = train[train['label']!=0]
train.drop_duplicates(subset=['sentence'], inplace=True)
train = train.dropna(how='any',axis=0) 
train = train.reset_index(drop=True)
len(train)

2420

In [19]:
# load wikipedia word embedding
wk = pd.read_csv('output/2.dropna_wiki_clean.csv')
wk.rename(columns={'Unnamed: 0': 'name'}, inplace=True)
wk_wordidx = word2idx(wk)
wk.head()

,name,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V292,V293,V294,V295,V296,V297,V298,V299,V300,V301
0,，,0.150785,0.120591,-0.220204,-0.044533,0.374161,0.044214,-0.392151,0.035214,0.300220,...,-0.187215,-0.150791,0.339811,0.069382,-0.323648,0.108729,0.065358,-0.532687,-0.066914,-0.100728
1,的,-0.251355,0.234742,-0.169728,0.026955,0.258850,0.028063,-0.498176,0.076507,0.151926,...,0.206175,-0.320072,0.206702,0.297621,-0.255238,0.076537,-0.021104,-0.527221,-0.229531,0.180781
2,。,0.207820,0.141425,-0.337281,0.158955,0.232709,0.008633,-0.401047,-0.088699,0.113330,...,0.108845,-0.074981,0.104715,-0.035879,-0.365303,-0.142196,0.015788,-0.555588,0.047561,-0.070897
3,、,0.020587,-0.472420,-0.441270,0.650659,0.406180,-0.197017,-0.230998,-0.362920,-0.372085,...,0.111319,-0.080049,0.768271,0.100696,-0.348324,-0.236327,0.031901,-0.870347,0.510523,0.065956
4,和,0.009246,0.187802,0.066590,-0.095002,0.193442,0.067781,-0.194179,-0.203283,-0.278398,...,0.300195,-0.004085,0.388441,-0.042360,-0.064346,-0.161472,0.136743,-0.496433,0.005538,0.038096


In [21]:
train_embedding = pd.DataFrame(index=range(0, len(train)), columns=range(0, 300))
test_embedding = pd.DataFrame(index=range(0, len(news)), columns=range(0, 300))

for i in range(0, len(train)):
    text = jieba.lcut(train.iloc[i,0])
    textidx = sen2idx(text, wk_wordidx)
    train_embedding.loc[i] = wk.loc[textidx,'V2':'V301'].mean(axis=0).values
    if i%1000==0:
        print(i)
        
for i in range(0, len(news)):
    text = jieba.lcut(news.iloc[i,0])
    textidx = sen2idx(text, wk_wordidx)
    test_embedding.loc[i] = wk.loc[textidx,'V2':'V301'].mean(axis=0).values
    if i%2500==0:
        print(i)

0
1000
2000
0
2500
5000
7500


In [22]:
train_lab = train['label'][train_embedding.isnull().any(axis=1)==False].values
train_embedding = train_embedding.loc[train_embedding.isnull().any(axis=1)==False]
test_lab = news['label'][test_embedding.isnull().any(axis=1)==False].values
news = news.loc[test_embedding.isnull().any(axis=1)==False]
test_embedding = test_embedding.loc[test_embedding.isnull().any(axis=1)==False]
train_wk = preprocessing.scale(train_embedding)
test_wk = preprocessing.scale(test_embedding)

In [23]:
clf = svm.SVC()
clf.fit(train_wk,train_lab)
y_pred=clf.predict(test_wk)
print("Accuracy:",metrics.accuracy_score(test_lab, y_pred))

Accuracy: 0.7703222967770322


In [24]:
for i in range(len(news)):
    news.iloc[i, 0] = ' ' + ' '.join(jieba.lcut(news.iloc[i, 0]))+' '
    
res = pd.DataFrame(index=range(5000), columns=range(20))
category = [range(0, 22), range(22, 37), range(37, 51), range(51, 66), range(66,83), 
            range(83, 95), range(95, 113), range(113, 121), range(121, 140), range(140, 182)]
for n, c in enumerate(category):
    counter = 0
    for w in c:
        counter += 1
        word = ' ' + words[w] + ' '
        id = [i for i in range(len(news)) if word in news.iloc[i, 0]]
        dif = (y_pred[id] - test_lab[id])
        start = 5000 - res.iloc[:,n].isnull().sum()
        end = start + len(dif.tolist())
        res.iloc[start:end, n] = dif.tolist()
        res.iloc[start:end, n+10] = counter
res.to_csv(r'output/svm_wk.csv', header=False, index=False)

In [25]:
clf = GaussianNB()
clf.fit(train_wk,train_lab)
y_pred=clf.predict(test_wk)
print("Accuracy:",metrics.accuracy_score(test_lab, y_pred))

Accuracy: 0.7622923770762292


In [26]:
res = pd.DataFrame(index=range(5000), columns=range(20))
category = [range(0, 22), range(22, 37), range(37, 51), range(51, 66), range(66,83), 
            range(83, 95), range(95, 113), range(113, 121), range(121, 140), range(140, 182)]
for n, c in enumerate(category):
    counter = 0
    for w in c:
        counter += 1
        word = ' ' + words[w] + ' '
        id = [i for i in range(len(news)) if word in news.iloc[i, 0]]
        dif = (y_pred[id] - test_lab[id])
        start = 5000 - res.iloc[:,n].isnull().sum()
        end = start + len(dif.tolist())
        res.iloc[start:end, n] = dif.tolist()
        res.iloc[start:end, n+10] = counter
res.to_csv(r'output/nb_wk.csv', header=False, index=False)

# People's Daily

In [28]:
news = pd.read_csv('auxiliary/Political_news_titles.csv')
news = news[news['label'].isnull()==False]
punct = ['......', '，', '--', '：',  '丨', '「', '」', '、', '——',  '-',  '_', '...', '·', '。',
        '【', '】', '《', '》', '/', '(图)', '(组图)', '[图]', '‧', '.', ':', ',', '（下）', '（上）', '（图）', '（组图）', ' ',
        '丨', '|', '(', ')', '（', '）', '[', ']', "'", '’', '‘', '“', '”', '%', '?', '!']
for i in range(len(news)):
    text = news.iloc[i,0]
    for p in punct:
      text = text.replace(p, "")
    text = re.sub('[a-zA-Z]','',text)
    text = re.sub("\d+", "", text)
    news.iloc[i,0] = text
news = news[news['label']!=0]
news.drop_duplicates(subset=['sentence'], inplace=True)
news = news.dropna(how='any',axis=0) 
news = news.reset_index(drop=True)
news["label"] = news["label"].astype(int)

train = pd.read_csv('auxiliary/TNEWS_subset.csv')
train = train[train['label'].isnull()==False]
for i in range(len(train)):
    text = train.iloc[i,0]
    for p in punct:
      text = text.replace(p, "")
    text = re.sub('[a-zA-Z]','',text)
    text = re.sub("\d+", "", text)
    train.iloc[i,0] = text
train = train[train['label']!=0]
train.drop_duplicates(subset=['sentence'], inplace=True)
train = train.dropna(how='any',axis=0) 
train = train.reset_index(drop=True)
len(train)

2420

In [29]:
# load wikipedia word embedding
rm = pd.read_csv('output/2.dropna_renmin_clean.csv')
rm.rename(columns={'Unnamed: 0': 'name'}, inplace=True)
rm_wordidx = word2idx(rm)
rm.head()

,name,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V292,V293,V294,V295,V296,V297,V298,V299,V300,V301
0,，,0.116763,-0.082260,-0.067078,-0.029883,-0.075887,-0.051286,0.007959,0.224768,0.252038,...,-0.239269,-0.022208,-0.039640,0.066611,0.041280,0.059769,0.046976,0.047563,0.033082,-0.003963
1,的,0.041535,-0.185500,-0.048225,-0.129857,0.002393,0.086562,-0.022853,0.230734,0.180895,...,-0.112451,-0.045284,-0.244256,0.056036,-0.069633,0.092941,0.028071,0.020061,-0.057477,-0.080065
2,。,0.210587,-0.163065,0.006192,-0.135545,0.038499,-0.120849,0.097238,0.116032,0.245585,...,-0.017580,-0.066345,-0.162462,0.048664,-0.147189,0.214738,0.141296,0.304065,-0.054951,-0.113204
3,、,-0.083906,-0.021279,-0.162081,-0.033575,-0.103414,-0.067491,0.035180,0.438456,0.488413,...,-0.512852,-0.018737,-0.279197,0.263514,-0.043667,0.074782,0.435261,0.270067,-0.008637,-0.186420
4,和,-0.034841,-0.225123,0.103976,0.091373,-0.090944,-0.110397,0.221433,0.327579,0.162576,...,-0.353931,-0.056001,-0.220450,0.217702,-0.156648,-0.145465,0.147922,0.195808,-0.243734,-0.213002


In [30]:
train_embedding = pd.DataFrame(index=range(0, len(train)), columns=range(0, 300))
test_embedding = pd.DataFrame(index=range(0, len(news)), columns=range(0, 300))

for i in range(0, len(train)):
    text = jieba.lcut(train.iloc[i,0])
    textidx = sen2idx(text, rm_wordidx)
    train_embedding.loc[i] = rm.loc[textidx,'V2':'V301'].mean(axis=0).values
    if i%1000==0:
        print(i)
        
for i in range(0, len(news)):
    text = jieba.lcut(news.iloc[i,0])
    textidx = sen2idx(text, rm_wordidx)
    test_embedding.loc[i] = rm.loc[textidx,'V2':'V301'].mean(axis=0).values
    if i%2500==0:
        print(i)

0
1000
2000
0
2500
5000
7500


In [31]:
train_lab = train['label'][train_embedding.isnull().any(axis=1)==False].values
train_embedding = train_embedding.loc[train_embedding.isnull().any(axis=1)==False]
test_lab = news['label'][test_embedding.isnull().any(axis=1)==False].values
news = news.loc[test_embedding.isnull().any(axis=1)==False]
test_embedding = test_embedding.loc[test_embedding.isnull().any(axis=1)==False]
train_rm = preprocessing.scale(train_embedding)
test_rm = preprocessing.scale(test_embedding)

In [32]:
clf = svm.SVC()
clf.fit(train_rm,train_lab)
y_pred=clf.predict(test_rm)
print("Accuracy:",metrics.accuracy_score(test_lab, y_pred))

Accuracy: 0.7708722912770872


In [33]:
for i in range(len(news)):
    news.iloc[i, 0] = ' ' + ' '.join(jieba.lcut(news.iloc[i, 0]))+' '
    
res = pd.DataFrame(index=range(5000), columns=range(20))
category = [range(0, 22), range(22, 37), range(37, 51), range(51, 66), range(66,83), 
            range(83, 95), range(95, 113), range(113, 121), range(121, 140), range(140, 182)]
for n, c in enumerate(category):
    counter = 0
    for w in c:
        counter += 1
        word = ' ' + words[w] + ' '
        id = [i for i in range(len(news)) if word in news.iloc[i, 0]]
        dif = (y_pred[id] - test_lab[id])
        start = 5000 - res.iloc[:,n].isnull().sum()
        end = start + len(dif.tolist())
        res.iloc[start:end, n] = dif.tolist()
        res.iloc[start:end, n+10] = counter
res.to_csv(r'output/svm_rm.csv', header=False, index=False)

In [34]:
clf = GaussianNB()
clf.fit(train_rm,train_lab)
y_pred=clf.predict(test_rm)
print("Accuracy:",metrics.accuracy_score(test_lab, y_pred))

Accuracy: 0.7802221977780223


In [35]:
res = pd.DataFrame(index=range(5000), columns=range(20))
category = [range(0, 22), range(22, 37), range(37, 51), range(51, 66), range(66,83), 
            range(83, 95), range(95, 113), range(113, 121), range(121, 140), range(140, 182)]
for n, c in enumerate(category):
    counter = 0
    for w in c:
        counter += 1
        word = ' ' + words[w] + ' '
        id = [i for i in range(len(news)) if word in news.iloc[i, 0]]
        dif = (y_pred[id] - test_lab[id])
        start = 5000 - res.iloc[:,n].isnull().sum()
        end = start + len(dif.tolist())
        res.iloc[start:end, n] = dif.tolist()
        res.iloc[start:end, n+10] = counter
res.to_csv(r'output/nb_rm.csv', header=False, index=False)